In [11]:
import os
import sys
import datetime
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window, WindowSpec
from pyspark.sql.functions import *
import pyspark.sql.functions as func

spark_framework_user = "sonia.pasquini"


os.environ["SPARK_HOME"] = "/usr/local/spark-2.1.0-bin-hadoop2.7/"
os.environ["SPARK_HOME_VERSION"] = "2.1.0"
os.environ["HADOOP_CONF_DIR"] = "/opt/hadoop/"
os.environ["SPARK_USER"] = spark_framework_user
#os.environ["PACKAGES"] = "com.databricks:spark-cassandra-connector_2.11-2.0.2.jar"
#os.environ["PACKAGES"] = "com.databricks:spark-csv_2.10:1.5.0"
#os.environ["PACKAGES"] = "com.databricks:spark-avro_2.11:3.2.0"
#os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages " + os.environ["PACKAGES"] + " pyspark-shell"

## impostare le configurazioni
config = SparkConf()\
        .setAppName(spark_framework_user+".Jupyter")\
        .setMaster("mesos://leader.mesos:5050")
  
 
config.set("spark.mesos.executor.docker.image", "artifacts.slu.bskyb.com:5003/mesosphere/spark:1.0.9-2.1.0-1-hadoop-2.7")
config.set("spark.mesos.executor.home", "/opt/spark/dist")
config.set("spark.executor.cores", "2")
config.set("spark.executor.memory", "15G")
config.set("spark.executor.heartbeat", "60s")
config.set("spark.cores.max", "10")
config.set("spark.driver.maxResultSize", "40G")
config.set("spark.driver.memory", "6G")
config.set("spark.driver.cores", "2")
#config.set("spark.cassandra.connection.host", "node-0-server.cassandra.mesos")
#config.set("spark.cassandra.connection.port", "9042")
#config.set("spark.cassandra.input.consistency.level","TWO")

config.set("spark.mesos.driver.failoverTimeout","5.0")
config.set("spark.ui.port", "22000")
# spark.ui.port = 22001

# inizializzare lo spark context
sc = SparkContext(conf = config)

# inizializzare 
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=sonia.pasquini.Jupyter, master=mesos://leader.mesos:5050) created by __init__ at <ipython-input-2-d7834dccfba5>:44 

In [53]:
path = "/user/stefano.mazzucca/new_kpi_digital/score_finale_ext_id.parquet"

id_digital = spark.read.parquet(path)

In [79]:
id_digital.count()

2692058

In [80]:
id_digital.distinct().count()

2622817

In [54]:
id_digital.printSchema()

root
 |-- external_id: string (nullable = true)
 |-- decile_kpi_digital: double (nullable = true)



In [55]:
comp_contatto = spark.read.csv("/user/sonia.pasquini/WORK_COMP_CONTATTO_FILTER1.csv", header=True, sep= ";")

In [56]:
comp_contatto.printSchema()

root
 |-- DAT_INI_VAL: string (nullable = true)
 |-- COD_CLIENTE_FRUITORE: string (nullable = true)
 |-- COD_CONTRATTO: string (nullable = true)
 |-- COD_CLIENTE_CIFRATO: string (nullable = true)
 |-- FASCIA_AGEING: string (nullable = true)
 |-- des_decoder_principale: string (nullable = true)
 |-- DES_STATO_BUSINESS: string (nullable = true)
 |-- num_gestite_om_360d: string (nullable = true)
 |-- num_accessi_APP: string (nullable = true)
 |-- num_accessi_WEB: string (nullable = true)
 |-- num_contatti_BO: string (nullable = true)
 |-- num_contatti_CHAT: string (nullable = true)
 |-- num_contatti_IVR: string (nullable = true)
 |-- comp_contatto: string (nullable = true)
 |-- COD_SESSO_CLIENTE_FRU: string (nullable = true)
 |-- FLG_SKYGO: string (nullable = true)
 |-- DES_AREA_NIELSEN_FRU: string (nullable = true)
 |-- DES_FORMATO_FATTURA: string (nullable = true)
 |-- DAT_NASCITA_CLIENTE_FRU: string (nullable = true)
 |-- DES_AMBIENTE_SERVIZIO: string (nullable = true)
 |-- data_iscriz

In [57]:
#quanti contratti ho al 01feb2019 (contratto_sto di gennaio19)
comp_contatto.select("COD_CONTRATTO", "COD_CLIENTE_FRUITORE", "COD_CLIENTE_CIFRATO").distinct().count()

4606458

In [58]:
#quanti clienti ho al 01feb2019 (contratto_sto di gennaio19)
comp_contatto.select("COD_CLIENTE_CIFRATO").distinct().count()

4469087

In [59]:
#faccio una distinct sulla lista di stefano
id_digital_distinct = id_digital.distinct()

In [60]:
# join con le info di contratto
digital_contatto = id_digital_distinct.join(comp_contatto, id_digital_distinct["external_id"] == comp_contatto["COD_CLIENTE_CIFRATO"], "left")

In [67]:
# conto quanti external id sono riuscita a matchare con i contratti
digital_contatto.filter("COD_CONTRATTO is not null ").count()

2716489

In [66]:
# conto quanti external ID  NON sono riuscita a matchare con i contratti
digital_contatto.filter("COD_CONTRATTO is null ").count()

5787

In [69]:
##  numero di contratti per external_id
count = digital_contatto.filter("COD_CONTRATTO is not null ").groupBy("external_id").agg({"COD_CONTRATTO": "count"})

In [76]:
# distribuzione del n° di contratti a cliente 

count.groupBy("count(COD_CONTRATTO)").agg({"count(COD_CONTRATTO)": "count"}).orderBy(desc("count(count(COD_CONTRATTO))")).show(45)

+--------------------+---------------------------+
|count(COD_CONTRATTO)|count(count(COD_CONTRATTO))|
+--------------------+---------------------------+
|                   1|                    2523037|
|                   2|                      90803|
|                   3|                       2581|
|                   4|                        365|
|                   5|                        118|
|                   6|                         34|
|                   8|                         19|
|                   7|                         19|
|                   9|                          8|
|                  12|                          6|
|                  11|                          6|
|                  15|                          5|
|                  13|                          5|
|                  16|                          3|
|                  10|                          2|
|                  19|                          2|
|                  18|         